In [ ]:
import pytorch_lightning as pl
import torch as th
import ChemtabUQ

In [ ]:
ckpt_path='../CT_logs_Mu/MAPE-PCA-CT/version_13420071/checkpoints/epoch=10891-step=10892.ckpt'
PL_module = ChemtabUQ.FFRegressor.load_from_checkpoint(ckpt_path, input_size=25)
print(PL_module)

In [ ]:
# from here: https://stackoverflow.com/questions/76839366/tf-rep-export-graphtf-model-path-keyerror-input-1
def convert_names_workaround(onnx_model):
    from onnx import helper
    
    # Define a mapping from old names to new names
    name_map = {"input.1": "input_1"}
    
    # Initialize a list to hold the new inputs
    new_inputs = []
    
    # Iterate over the inputs and change their names if needed
    for inp in onnx_model.graph.input:
        if inp.name in name_map:
            # Create a new ValueInfoProto with the new name
            new_inp = helper.make_tensor_value_info(name_map[inp.name],
                                                    inp.type.tensor_type.elem_type,
                                                    [dim.dim_value for dim in inp.type.tensor_type.shape.dim])
            new_inputs.append(new_inp)
        else:
            new_inputs.append(inp)
    
    # Clear the old inputs and add the new ones
    onnx_model.graph.ClearField("input")
    onnx_model.graph.input.extend(new_inputs)
    
    # Go through all nodes in the model and replace the old input name with the new one
    for node in onnx_model.graph.node:
        for i, input_name in enumerate(node.input):
            if input_name in name_map:
                node.input[i] = name_map[input_name]
    return onnx_model

In [ ]:
import os
from glob import glob
print(os.getcwd())
print(glob('../CT_logs_Mu/MAPE-PCA-CT/*/checkpoints/*'))

In [ ]:
onnx_model_path = os.path.dirname(ckpt_path)+'/model.onnx'
print(onnx_model_path)
PL_module.to_onnx(onnx_model_path)

In [ ]:
%pdb on
import onnx
from onnx_tf.backend import prepare
onnx_model = convert_names_workaround(onnx.load(onnx_model_path))
tf_rep = prepare(onnx_model)
tf_model_path = os.path.dirname(ckpt_path)+'/model_TF'
tf_rep.export_graph(tf_model_path)
#print(onnx_model)

In [ ]:
%pdb off
import tensorflow as tf
import tensorflow.keras as keras
x=tf.zeros(16,25)
keras_version = keras.models.load_model(tf_model_path)
keras_version()

In [ ]:
res = os.popen(f'onnx-tf convert -i {onnx_model_path} -o output.pb')
print(res)
#%onnx-tf convert -t tf -i ../CT_logs_Mu/MAPE-PCA-CT/version_13420071/checkpoints/model.onnx -o output.pb

In [ ]:
"""%pip install onnx
%pip install onnx_tf
%pip install tensorflow_probability"""

%pip uninstall -y tensorflow
%pip install tensorflow
%pip uninstall -y tensorflow_probability
%pip install tensorflow_probability==0.19.0

In [ ]:
%pip show tensorflow_probability

In [ ]:
%pip install pytorch-lightning

In [ ]:
!pip install pandas scikit-learn

In [ ]:
import sklearn

In [ ]:
%pip install scikit-learn

In [ ]:
%pip install onnx

In [ ]:
%pip install tensorflow_probability
%pip show tensorflow_probability
%pip show scikit-learn
%pip show pytorch-lightning

In [ ]:
?tf_rep.export_graph

In [ ]:
!python --version

In [ ]:
!pip show onnx

In [ ]:
!pip show onnx-tf